# Exploring GBMCMC Outputs in Jupyter & PANDAS

## Introduction
This notebook provides a skeleton example for exploring processed outputs of the GBMCMC code.  To use this notebook, GBMCMC outputs must first be processed through the catalog maker step and then into a set of HDF5 files using the script `build_pandas_catalog.py` located in `/scripts`.  For further information on `build_pandas_catalog.py`, please consult the inline help.

### The Data
The output of `build_pandas_catalog.py` is a set of HDF5 files containing several PANDAS dataframes.  These consist of a primary file (e.g. `mycatalog.h5`) as well as one or more auxilliary files with related names (e.g. `mycatalog_chains_100s.h5`).  The primary file contains three dataframes: `metadata` contains information about the catalog, `detections` contains point-estimates of the catalog entries, and `spectrum` which contains frequency-series information about the data, model, residuals, noise estimates, etc.  The auxilliary files contain MCMC parameter chains for the identified entries in the catalog, allowing for deeper-dive analysis into a particular source.  For size reasons, they are grouped by frequency segment in the GBMCMC outputs with a separate file for each 100 frequency segments.

## Initialization
Load the various modules/packages required for this notebook and define local functions

### Modules

In [ ]:
# Import modules
import os
import glob
import pandas as pd
import corner as corner
import numpy as np    
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import matplotlib.colors as colors
import matplotlib.cm as cm
import lisacattools as lisacat
#import corner
%matplotlib inline

## Locate catalogs and examine meta data
Each catalog corresponds to a particular observation duration and is contained in a single HDF5 file.  The HDF5 file contains several pandas data frames, one of which is called _metadata_ and it contains metadata about the catalog.  Here we search through the directory and get the list of catalogs and their meta data.  Here the metadata is the observation time and the parent catalog (used for tracing the lineage of particular sources from catalog to catalog)

In [ ]:
plotPath = ''  # INSERT YOUR DESIRED DIRECTORY FOR PLOT OUTPUTS HERE
catFiles = [ '',''] #INSERT LIST OF ONE OR MORE PRIMARY CATALOG FILES from build_pandas_catalogs.py
dfs = list()

# Read metadata from each catalog, store the location of the file on your local system, and sort by observation time
for catFile in catFiles:
    df = pd.read_hdf(catFile, key = 'metadata')
    df['location'] = catFile
    dfs.append(df) 
meta  = pd.concat(dfs)
meta = meta.sort_values(by='Observation Time')
meta

### Using the source point estimates in a catalog
Each catalog also contains a list of _detections_ (candidate sources) with estimates for the parameters in a dataframe called `detections`. Each source is indexed by a name (which corresponds to the median frequency) and has a list of parameter estimates. In addition, the SNR, Bayesian evidence, and frequency segment (internal parameter from the GBMCMC search code) are included for each detection. For catalogs with a parent catalog, a source identified as a parent source in that catalog is listed, if present.

In [ ]:
# load the point estimates from the `detections` dataframe
catIdx = 0  # Select which catalog from your list you want to examine
catPath = os.path.split(catFiles[catIdx])[0]
catRoot = os.path.split(os.path.splitext(catFiles[catIdx])[0])[1] # string for naming plots
catFile = meta.iloc[catIdx]['location']
cat = pd.read_hdf(catFile, key='detections')
cat[['SNR','parent','Frequency','Frequency Derivative','Amplitude','Ecliptic Longitude','Ecliptic Latitude','Polarization','Inclination','Initial Phase']].head()

In [ ]:
# The describe() method provides some basic statistics on the catalog, including number of sources, parameter statistics, etc.
cat.describe()

In [ ]:
# Plots can be made directly from the data, such as this histogram of SNRs
fig = plt.figure(figsize = [20,6],dpi=100)
ax = plt.axes()
cat['SNR'].plot(kind='hist', ax = ax,grid = True, bins=100,figsize=[12,8])
plt.xlabel('SNR',fontsize=12)
plt.ylabel('Count',fontsize=12)
plt.title('SNR Distribution for %i sources in %s' % (len(cat),meta.index[catIdx]),fontsize=12)
fig.savefig('%s/%s_SNR_hist.png' % (plotPath,catRoot))

In [ ]:
# Another example of is a 2-D scatter plot, such as frequency-amplitude 
fig = plt.figure(figsize=(12, 6), dpi = 100)
ax = plt.axes()

plt.yscale('log')
plt.xscale('log')
plt.xlim([1e-4,1e-1])
plt.ylim([1e-24,1e-20])
ax.tick_params(axis = 'both', which = 'major', labelsize = 12)

plt.title('Point estimates for %i sources found in catalog %s' % (len(cat), meta.index[catIdx]),fontsize=18)

cNorm = colors.LogNorm(vmin=cat['SNR'].min(), vmax=cat['SNR'].max()) #re-wrapping normalization
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=plt.cm.get_cmap('cool'))

cat.plot(
    kind='scatter', 
    x='Frequency', 
    y='Amplitude',  
    marker = '.',
    c = scalarMap.to_rgba(np.array(cat['SNR'])),
    ax = ax);

ax.grid()
plt.xlabel('Frequency [Hz]',fontsize=14)
plt.ylabel('Strain Amplitude',fontsize=14)

f = np.logspace(-4,0,512)
ax.plot(f,lisacat.getSciRD(f,np.float(meta.iloc[catIdx]['Observation Time'])), color='k')

ax.legend(['Instrument Sensitivity','resolved GBs'],fontsize=14)

cbar = fig.colorbar(scalarMap)
cbar.set_label('SNR',fontsize=14)
plt.show()
fig.savefig('%s/%s_strain.png' % (plotPath,catRoot))

## Frequency-series data

In [ ]:
# load the HDF5 file containing the Pandas data frame with all of the frequency-series data from the GBMCMC run. 
# This dataframe was constructed by looping through all of the output directories of the GBMCMC run
catIdx = 0  # Select which catalog from your list you want to examine
catPath = os.path.split(catFiles[catIdx])[0]
catRoot = os.path.split(os.path.splitext(catFiles[catIdx])[0])[1] # string for naming plots
catFile = meta.iloc[catIdx]['location']
fdf = pd.read_hdf(catFile, key='spectrum')
fdf.sort_values('Frequency',inplace=True)
fdf.describe()

In [ ]:
# input data is in real/imag.  Convert to power so we can compare with the noise
fdf['A data power']= np.power(fdf['Real A data'],2)+np.power(fdf['Imag A data'],2)
fdf['E data power']= np.power(fdf['Real E data'],2)+np.power(fdf['Imag E data'],2)

In [ ]:
# plot power residual for A channel
fig = plt.figure(figsize = [12,8],dpi=100)
ax = plt.axes()
fdf.plot(x='Frequency',y='A data power',ax = ax)
fdf.plot(x='Frequency',y='Median A residual',ax = ax)
fdf.plot(x='Frequency',y='Median AE noise',ax=ax)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Fractional Frequency PSD [1/Hz]')
plt.yscale('log')
plt.xscale('log')
plt.xlim([1e-4,3e-2])
ax.grid(True)
plt.title('Power spectra for TDI A channel in %s' % meta.index[catIdx],fontsize=18)
fig.savefig('%s/%s_A_power.png' % (plotPath,catRoot))

# plot power residual for E channel
fig = plt.figure(figsize = [12,8],dpi=100)
ax = plt.axes()
fdf.plot(x='Frequency',y='E data power',ax = ax)
fdf.plot(x='Frequency',y='Median E residual',ax = ax)
fdf.plot(x='Frequency',y='Median AE noise',ax=ax)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Fractional Frequency PSD [1/Hz]')
plt.yscale('log')
plt.xscale('log')
plt.xlim([1e-4,3e-2])
ax.grid(True)
plt.title('Power spectra for TDI E channel in %s' % meta.index[catIdx],fontsize=18)
fig.savefig('%s/%s_E_power.png' % (plotPath,catRoot))

### Working with Individual Chain Data
Each of the auxilliary HDF5 files contains the MCMC chains (burn-in removed) for each of the detected sources as pandas data frames. They are identified in the HDF5 file by the key `<SOURCENAME>_chain` where `<SOURCENAME>` is the name of the source in the catalog of detections

In [ ]:
# pick out the median SNR source
catIdx = 0  # Select which catalog from your list you want to examine
catPath = os.path.split(catFiles[catIdx])[0]
catRoot = os.path.split(os.path.splitext(catFiles[catIdx])[0])[1] # string for naming plots
catFile = meta.iloc[catIdx]['location']
cat = pd.read_hdf(catFile, key='detections')
sourceIdx = cat.index.values[np.argmin(np.abs(np.array(cat['SNR'])-cat['SNR'].median()))]
cat.loc[[sourceIdx]]

In [ ]:
# read in the chain samples for this source
source = lisacat.getChain(cat,sourceIdx,catPath)
source.head()

In [ ]:
# Statistics on the chain samples provide some view of the parameter distributions
source.describe()

In [ ]:
# Plotting the chains versus MCMC sample gives a quick picture as to whether they are converged, etc. 
source[['Frequency','Amplitude','Frequency Derivative','Ecliptic Longitude','Ecliptic Latitude','Inclination','Initial Phase','Polarization']].plot(
    kind='line',
    subplots = True, 
    layout=[4,2], 
    figsize=[12,10]);
fig = plt.gcf()
fig.suptitle('MCMC chains for %s' % sourceIdx)
fig.savefig('%s/%s_chains.png' % (plotPath,sourceIdx))

In [ ]:
# You can get a correlation table as well to pick out correlations in parameters 
source.corr()

In [ ]:
# Or you can do this in a corner plot
fig = corner.corner(
    source[['Frequency',
            'Amplitude',
            'Frequency Derivative',
            'Ecliptic Longitude',
            'Ecliptic Latitude']],
            quantiles=(0.16, 0.84));
fig.suptitle('Corner plot for %s' % sourceIdx)
fig.savefig('%s/%s_corner.png' % (plotPath,sourceIdx))

In [ ]:
# do a comparison of the error ellipse and points and make a nice scatter plot
lisacat.getGalcoord(source) # convert to galactic coordinates
fig = plt.figure(figsize=(8, 6), dpi = 100)
ax = plt.axes()
ax.grid()

a =  lisacat.ellipse_area(source[['Galactic Longitude','Galactic Latitude']])
source.plot(kind='scatter',x='Galactic Longitude',y='Galactic Latitude',ax=ax,grid=True,marker='.')
lisacat.confidence_ellipse(source[['Galactic Longitude','Galactic Latitude']], 
                       ax, 
                       n_std = 1.0, 
                       edgecolor='orange',
                       linewidth = 2.0)
lisacat.confidence_ellipse(source[['Galactic Longitude','Galactic Latitude']], 
                       ax, 
                       n_std = 2.0, 
                       edgecolor='orange',
                       linestyle = '--',
                       linewidth = 2.0)
lisacat.confidence_ellipse(source[['Galactic Longitude','Galactic Latitude']], 
                       ax, 
                       n_std = 3.0, 
                       edgecolor='orange',
                       linestyle=':',
                       linewidth = 2.0)

plt.xlabel('Galactic Longitude [deg]')
plt.ylabel('Galactic Latitude [deg]')
plt.title('Sky Localization for %s: 90%% region of %3.1f deg^2' % (sourceIdx,a))
ax.legend(('1 sigma','2 sigma', '3 sigma','MCMC samples'))
#fig.savefig('%s/%s_localization.png' % (plotPath,sourceIdx))

### Adding derived parameters from the chains
In some cases, it might be desirable to estimate some addiitonal parameter from the chains, such as luminosity distance

In [ ]:
# The get_DL function defined in lisacattools computes luminosity distance for each sample in the chain and adds it as a column
lisacat.get_DL(source)
source.describe()

In [ ]:
# histogram of luminosity distance
fig = plt.figure(figsize = [10,6],dpi=100)
ax = plt.axes()
source['Luminosity Distance'].hist(bins=50,ax=ax,grid=True)
plt.xlabel('DL [kpc]')
plt.ylabel('Count')
plt.title('Luminosity distance distribution for %s' % sourceIdx)
fig.savefig('%s/%s_Dlhist.png' % (plotPath,sourceIdx))

In [ ]:
# Adding Cartesian coordinates and looking at the subpart of parameter space relative to localization
source['X']=source['Luminosity Distance']*np.cos(source['Galactic Latitude'])*np.cos(source['Galactic Longitude'])
source['Y']=source['Luminosity Distance']*np.cos(source['Galactic Latitude'])*np.sin(source['Galactic Longitude'])
source['Z'] = source['Luminosity Distance']*np.sin(source['Galactic Latitude'])
fig = corner.corner(
    source[['X',
            'Y',
            'Z']],
            quantiles=(0.16, 0.84));
fig.suptitle('Cartesian Localization Corner plot for %s' % sourceIdx)
fig.savefig('%s/%s_local_corner_cart.png' % (plotPath,sourceIdx))

### Extending the catalog - sky area
In some cases, it might be useful to add additional columns to the catalog from derived parameters.  Here we use sky localization as an example

In [ ]:
catIdx = 0  # Select which catalog from your list you want to examine
catPath = os.path.split(catFiles[catIdx])[0]
catRoot = os.path.split(os.path.splitext(catFiles[catIdx])[0])[1] # string for naming plots
catFile = meta.iloc[catIdx]['location']
cat = pd.read_hdf(catFile, key='detections')
# loop through all of the cases, get the area, and add as a column to the catalog 
areas = np.empty(len(cat))
sources = list(cat.index)
for idx,source in enumerate(sources):
    df = lisacat.getChain(cat,source,catPath)
    lisacat.getGalcoord(df)
    areas[idx] = lisacat.ellipse_area(df[['Galactic Longitude','Galactic Latitude']])

cat.insert(len(cat.columns),'Sky Area',areas,True)
cat.head()

In [ ]:
# Histogram of Sky Areas
fig = plt.figure(figsize = [20,6],dpi=100)
ax = plt.axes()
cat['Sky Area'].plot(kind='hist', ax = ax,grid = True, bins=100,figsize=[12,8])
plt.xlabel('Sky Area[deg^2]')
plt.ylabel('Count')
plt.title('90%% Sky Area Distribution for %i sources in %s' % (len(cat), meta.index[catIdx]))
fig.savefig('%s/%s_skyarea_hist.png' % (plotPath,catRoot))

In [ ]:
# Make a cut to find "well-localized" events
athresh = 1000;
cat_loc = cat[(cat['Sky Area']<athresh)]
cat_loc.describe()

In [ ]:
# Plot the elipses for all the well-localized sources in the catalog
# set up the figure
fig = plt.figure(figsize=(12, 6), dpi = 100)
ax = plt.axes()#projection='astro degrees mollweide')
ax.grid()

plt.xlim(-180,180)
plt.ylim(-90,90)
plt.xlabel('Galactic Longitude')
plt.ylabel('Galactic Latitude')
 
cNorm = colors.Normalize(vmin=cat_loc['Frequency'].min()*1e3, vmax=cat_loc['Frequency'].max()*1e3) #re-wrapping normalization
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=plt.cm.get_cmap('gist_rainbow'))

sources = list(cat_loc.index)
for source in sources:
    df = lisacat.getChain(cat,source,catPath)
    lisacat.getGalcoord(df)
    #a = lisacat.ellipse_area(df[['Galactic Longitude','Galactic Latitude']])
    m = np.array(df[['Galactic Longitude','Galactic Latitude']].mean())
    lisacat.confidence_ellipse(df[['Galactic Longitude','Galactic Latitude']], 
                       ax, 
                       n_std = 1.0, 
                       edgecolor=scalarMap.to_rgba(1e3*np.array(cat_loc.loc[source].Frequency)),
                       linewidth = 1.0)
                       
                       
plt.title('Localization for %i sources in catalog %s with sky area < %i deg^2' % (len(cat_loc),meta.index[catIdx],athresh))
cbar = fig.colorbar(scalarMap)
cbar.set_label('Frequency [mHz]')
plt.show()
fig.savefig('%s/%s_skymap_good.png' % (plotPath,catRoot))


## Comparing Catalogs
It is possible to load multiple catalogs (e.g. from different epochs) and cross-compare

In [ ]:
# Here we load each catalog into a dictionary using the catalog names as a key
allCats = dict()
for idx in range(0,len(meta)):
    allCats[meta.index[idx]]= pd.read_hdf(meta.iloc[idx]['location'], key='detections')

In [ ]:
# You can access a particular catalog by its name
allCats['INSERT_NAME_HERE'].describe() 

In [ ]:
# Comparison of SNRs
fig = plt.figure(figsize = [8,6],dpi=100)
ax = fig.add_subplot(111)
pcolors = ['r','b','g','y']
pltfile = 'SNR'
for cnt,idx in enumerate(list(meta.index)):
    allCats[idx]['SNR'].hist(bins=100,color=pcolors[cnt], ax=ax,grid=True,histtype='stepfilled', alpha=0.2,density=False)
    pltfile = ('%s_%s' % (pltfile,idx))
    
ax.set_xlabel('SNR')
ax.set_ylabel('Count')
plt.legend(meta.index)
ax.set_title('SNR comparison between catalogs')
fig.savefig('%s/%s.png' % (plotPath,pltfile))

In [ ]:
# Comparison on frequency/amplitude plane 
fig = plt.figure(figsize=(12, 10), dpi = 100)
ax = plt.axes()

plt.yscale('log')
plt.xscale('log')
plt.xlim([1e-4,1e-1])
plt.ylim([1e-24,1e-20])
ax.tick_params(axis = 'both', which = 'major', labelsize = 12)

plt.title('Comparison of catalogs',fontsize=18)

pcolors = ['r','b','g','y']
pltfile = 'sources'
for cnt,idx in enumerate(list(meta.index)):
    allCats[idx].plot(
        kind='scatter', 
        x='Frequency', 
        y='Amplitude',  
        marker = '.',
        color = pcolors[cnt],
        label = idx,
        ax = ax);
    
    
    ax.plot(f,lisacat.getSciRD(f,np.array(meta.iloc[cnt]['Observation Time'])), color=pcolors[cnt],linestyle='--', label = ('%0.0f mo sensitivity' % (np.array(meta.iloc[cnt]['Observation Time'])/(30.5*86400))))

    pltfile = ('%s_%s' % (pltfile, idx))



ax.grid()
plt.xlabel('Frequency [Hz]',fontsize=14)
plt.ylabel('Strain Amplitude',fontsize=14)

f = np.logspace(-4,0,512)

ax.legend(fontsize=14)

fig.savefig('%s/%s.png' % (plotPath,pltfile))

## Comparing sources across catalogs
In some cases, it may be interesting to compare sources across catalogs. For example, we might want to see how parameter estimates evolved as we got more data

In [ ]:
catIdx = 1  # Select the catalog that has the source you want to examine
catPath = os.path.split(catFiles[catIdx])[0]
catRoot = os.path.split(os.path.splitext(catFiles[catIdx])[0])[1] # string for naming plots
catFile = meta.iloc[catIdx]['location']
cat = pd.read_hdf(catFile, key='detections')
# pick out the subset of sources that have identified parents
cat = cat[cat['parent']!='']

In [ ]:
# Pick out a source to examine, say the one with the median SNR
sourceIdx = cat.index.values[np.argmin(np.abs(np.array(cat['SNR'])-cat['SNR'].median()))]
cat.loc[[sourceIdx]]

In [ ]:
# Now we load the source data
source = lisacat.getChain(cat,sourceIdx,catPath)
source.describe()

In [ ]:
# Now we load the parent data from the parent catalog
parentCatFile = meta.loc[meta.iloc[catIdx]['parent']]['location']
parentCat = pd.read_hdf(parentCatFile, key='detections')
parentIdx = cat.loc[sourceIdx,'parent']
parentCat.loc[[parentIdx]]

In [ ]:
# and finally load the chain data from the parent catalog
parentSource = lisacat.getChain(parentCat,parentIdx, catPath)
parentSource.describe()

In [ ]:
# Comparison Corner plots
# since there are so many parmeters, we break into intrinsic and extrinsic

# intrinsic parameters
fig = plt.figure(figsize=(10, 10), dpi = 100)
inParams = ['Frequency','Frequency Derivative', 'Amplitude']
corner.corner(parentSource[inParams], fig=fig,color='red', plot_datapoints=False, fill_contours=True, bins=50, smooth=1.0, levels=[0.68,0.95], label_kwargs={"fontsize": 12});
corner.corner(source[inParams], fig=fig,color='blue', plot_datapoints=False, fill_contours=True, bins=50, smooth=1.0, levels=[0.68,0.95], label_kwargs={"fontsize": 12});
fig.suptitle('Comparison of intrinsic parameters between %s and parent %s' % (sourceIdx,parentIdx))
fig.savefig('%s/%s_vs_%s_intrinsic.png' % (plotPath,sourceIdx, parentIdx))

# extrinsic parameters
fig = plt.figure(figsize=(10, 10), dpi = 100)
extParams = ['Ecliptic Latitude','Ecliptic Longitude','Inclination','Initial Phase','Polarization']
corner.corner(parentSource[extParams], fig=fig,color='red', plot_datapoints=False, fill_contours=True, bins=50, smooth=1.0, levels=[0.68,0.95], label_kwargs={"fontsize": 12});
corner.corner(source[extParams], fig=fig,color='blue', plot_datapoints=False, fill_contours=True, bins=50, smooth=1.0, levels=[0.68,0.95], label_kwargs={"fontsize": 12});
fig.suptitle('Comparison of extrinsic parameters between %s and parent %s' % (sourceIdx,parentIdx))
fig.savefig('%s/%s_vs_%s_extrinsic.png' % (plotPath,sourceIdx, parentIdx))
